#### What are you trying to do in this notebook?
For this challenge, we will be predicting bacteria species based on repeated lossy measurements of DNA snippets. Snippets of length 10 are analyzed using Raman spectroscopy that calculates the histogram of bases in the snippet.

#### Why are you trying it?
The DNA segment ATATGGCCTT becomes A2T4G2C2.

Each row of data contains a spectrum of histograms generated by repeated measurements of a sample, each row containing the output of all 286 histogram possibilities (e.g., A0T0G0C10 to A10T0G0C0), which then has a bias spectrum (of totally random ATGC) subtracted from the results.

The data (both train and test) also contains simulated measurement errors (of varying rates) for many of the samples, which makes the problem more challenging.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import plotly as py
from statistics import mean
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import seaborn as sns
import random
import time
import os

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import VotingClassifier
from tqdm.notebook import tqdm

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
from xgboost import XGBClassifier
from scipy.stats import mode
import warnings
warnings.simplefilter('ignore')

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")
train = train[0:30000]

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")

In [ ]:
target = 'target'
target_encoded = 'target_encoded'
features = [col for col in test.columns if 'id' not in col]

In [ ]:
le = LabelEncoder()
train[target_encoded] = le.fit_transform(train[target])

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
reduce_mem_usage(train)
reduce_mem_usage(test)

In [ ]:
paramsXGB = {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 400, 'min_child_weight': 2, 'gamma': 0, 'colsample_bytree': 0.6, 'subsample': 0.5,'booster': 'gbtree'}

In [ ]:
extratree_params = {
    'n_estimators' : 1000,
    'random_state' : 42,
    'verbose' : 1,
    'n_jobs' : -1
}
extratree_params1 = {
    'n_estimators' : 1000,
    'random_state' : 64,
    'verbose' : 1,
    'n_jobs' : -1
}

In [ ]:
xgb_model = XGBClassifier(**paramsXGB)
extratree_model = ExtraTreesClassifier(**extratree_params)
extratree_model1 = ExtraTreesClassifier(**extratree_params1)

In [ ]:
X=train[features]
y=train[target_encoded]
X_test = test[features]
del train,test

In [ ]:
pred = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (trn_idx, val_idx) in enumerate(tqdm(skf.split(X, y), total=5)):
    X_train = X.iloc[trn_idx]
    y_train = y.iloc[trn_idx]
    X_valid = X.iloc[val_idx]
    y_valid = y.iloc[val_idx]

    start = time.time()
    model = VotingClassifier(
            estimators = [
                ('xgb', xgb_model),
                ('extratree', extratree_model), 
                #('extratree1', extratree_model1)
            ],
            voting = 'soft',
            weights = [0.4, 0.6],
            n_jobs = -1
        )
    model.fit(X_train, y_train)

    pred.append(model.predict(X_test))

    elapsed = time.time() - start

In [ ]:
del model,X_train,X_valid

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

In [ ]:
pred_decoded = le.inverse_transform(mode(pred).mode[0])
submission[target] = pred_decoded
submission.to_csv("submission.csv", index=False)
submission

#### Did it work?
target column is the target variable which consists of 10 kinds of bacteria Streptococcus_pyogenes,Salmonella_enterica,Enterococcus_hirae, Escherichia_coli,Campylobacter_jejuni,Streptococcus_pneumoniae, Staphylococcus_aureus,Escherichia_fergusonii,Bacteroides_fragilis,Klebsiella_pneumoniae

Train dataset has 200,000 rows and 288 columns which contains286 features, 1 target variable target and 1 column of row_id

Test dataset has 100,000 rows and 287 columns which contains286 features with1 column of row_id
No missing values in train and test dataset.

#### What did you not understand about this process?
Well, everything provides in the competition data page. I've no problem while working on it. If you guys don't understand the thing that I'll do in this notebook then please comment on this notebook.

#### What else do you think you can try as part of this approach?
Classify 10 different bacteria species using data from a genomic analysis technique that has some data compression and data loss. In this technique, 10-mer snippets of DNA are sampled and analyzed to give the histogram of base count.